In [98]:
import pandas as pd
import numpy as np

In [99]:
#data_bkg = pd.read_csv('/home/debo/UTILITIES/ttbar_out.txt', header=None, skiprows=2,error_bad_lines=False)
data_bkg = pd.read_csv('/home/debo/UTILITIES/ttbar_out.txt', header=None, skiprows=2)
data_sig = pd.read_csv('/home/debo/UTILITIES/gluino_out.txt', header=None, skiprows=2)

In [100]:
print("data_bkg: ",data_bkg.shape)
print("data_sig: ",data_sig.shape)

('data_bkg: ', (2174419, 95))
('data_sig: ', (1053206, 113))


In [101]:
data_bkg = data_bkg.values
data_sig = data_sig.values

In [102]:
print("data_bkg: ",data_bkg.shape)
print("data_sig: ",data_sig.shape)

('data_bkg: ', (2174419, 95))
('data_sig: ', (1053206, 113))


In [103]:
data_bkg_pad = np.zeros((2174419,18))

In [104]:
data_bkg_padded = np.concatenate((data_bkg, data_bkg_pad),1)


In [105]:
data_bkg_padded.shape

(2174419, 113)

In [106]:
data_bkg_padded[0,:]

array([ 2.63127e+04, -2.24806e+00,  6.34814e+04,  6.89961e-01,
        1.45730e+00,  5.49048e+04, -7.20720e-01, -2.71409e+00,
        2.81850e+04,  1.89810e+00,  2.62792e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  8.84342e+04,  2.04707e+00, -5.21237e-01,
        3.96456e+04,  1.99887e+00, -1.93787e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.000

In [107]:
truth = np.array(([1] * data_sig.shape[0]) + ([0] * data_bkg_padded.shape[0]))

In [108]:
truth.shape

(3227625,)

In [109]:
truth_r = truth.reshape(3227625,1)
l = np.zeros((3227625,1))
n = l[np.where(truth==0)]
n.size

2174419

In [110]:
total_bk_sig = np.concatenate((data_sig, data_bkg_padded),0)

In [111]:
total_bk_sig.shape

(3227625, 113)

In [112]:
data_truth = np.concatenate((total_bk_sig, truth_r),1)

In [113]:
data_truth.shape

(3227625, 114)

In [114]:
np.random.shuffle(data_truth)

In [115]:
data_truth.shape

(3227625, 114)

In [116]:
data_f = data_truth[:,0:113]
truth_f= data_truth[:,113:114]

In [117]:
data_f.shape

(3227625, 113)

In [118]:
l = np.zeros((3227625, 1))

In [121]:
n = l[np.where(truth_f==0)]

In [124]:
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import KFold
import h5py

In [125]:
X_train, X_test, y_train, y_test = train_test_split(data_f, truth_f, train_size=0.75)

In [126]:
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

('y_train: ', (2420718, 1))
('y_test: ', (806907, 1))


In [128]:
l = np.zeros((806907,1))
n = l[np.where(y_test==0)]
n.size

544241

In [129]:
1630178+544241

2174419

In [130]:
Train_ = {
        'X' : X_train,
        'y' : y_train,
    }
Test_ = {
        'X' : X_test,
        'y' : y_test,
    }

In [131]:
import os

In [132]:
SAMPLE_PATH = os.path.join('/home/debo/forgit')

In [133]:
import deepdish.io as io

In [134]:
io.save(os.path.join(SAMPLE_PATH, 'DarkMachine' + 'Train' + '.h5'), Train_)

In [135]:
io.save(os.path.join(SAMPLE_PATH, 'DarkMachine' + 'Test' + '.h5'), Test_)